<a href="https://colab.research.google.com/github/shoond/portfolio/blob/main/LoanReplayments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loan Repayment Exercise

Goal: Minimize the risk of a borrower being unable to repay a loan.
* Predict "NotfullyPaid" using all the other variables as independent variables.

NotFullyPaid = 0 means loans paid back in full

NotFullyPaid = 1 means debt

Data:
* 9578 observations, each representing a 3-yr loan between May 2007 and Feb 2010.
* There are 14 variables in dataset.



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
import statsmodels.formula.api as smf

In [3]:
df_loans = pd.read_csv('https://raw.githubusercontent.com/amandeep0/IS451/main/data/loans.csv')

df_loans.describe()     #Peek at rows
df_loans.head()     #Peek at data
df_loans.dtypes

# There exists str values for column['Purpose'], we need to adjust to int values for our model.
# float64 vs int64 is okay, just a performance thing

CreditPolicy        int64
Purpose            object
IntRate           float64
Installment       float64
LogAnnualInc      float64
Dti               float64
Fico                int64
DaysWithCrLine    float64
RevolBal            int64
RevolUtil         float64
InqLast6mths        int64
Delinq2yrs          int64
PubRec              int64
NotFullyPaid        int64
dtype: object

In [4]:
# Use pd.factorize() to assign integer values for df_loans['Purpose']

df_loans['Purpose'] = pd.factorize(df_loans['Purpose'])[0]

print(df_loans['Purpose'])

0       0
1       1
2       0
3       0
4       1
       ..
9573    2
9574    2
9575    0
9576    3
9577    0
Name: Purpose, Length: 9578, dtype: int64


# a

## a_i
* Split the dataset: 70/30 train/test
* What is the accuracy on the test set of a simple baseline model that predicts loans will be paid back in full (NotFullyPaid = 0)?
> 84.20%

In [5]:
train = 0.70

#Split the data
df_loans_train, df_loans_test = train_test_split(df_loans, test_size=(1-train), random_state=23)

print(df_loans_train.shape[0]/df_loans.shape[0])    #Confirm 70% train

print(df_loans_test.shape[0]/df_loans.shape[0])     #Confirm 30% test


0.6999373564418458
0.3000626435581541


In [6]:
print(df_loans_test['NotFullyPaid'].value_counts())

#Baseline says borrowers will pay back in full.

baseline = 2420/(2420+454)
print(baseline)

0    2420
1     454
Name: NotFullyPaid, dtype: int64
0.8420320111343076


#a_ii

Now, build a logistic regression model that predicts "NotFullyPaid" using all of the other variables as indepedent variables. Use the training set as the data for the model.

* Describe your model.
> The model is roughly 84% accurate in predicting loans not being paid. Although there exist a couple variables such a FICO that are inversely correlated to loans not being paid, where as increases in FICO would signify credit trustworthy and loans being successfully paid.

* Which of the independent variables are significant in your model?
> CreditPolicy, Purpose, IntRate, Installment, LogAnnualInc, Fico, InqLast6mths, PubRec

In [7]:
# We create a for loop to concat all the column names together in order to place into logit_reg function in the next step.
# We could copy and paste each one individually but this would be faster.

columns = [col for col in df_loans.columns if col!= 'NotFullyPaid']

joined = '+'.join(columns)

col_str = 'NotFullyPaid ~ ' + joined   # assign the concat string to 'reg_all'

print(col_str)

NotFullyPaid ~ CreditPolicy+Purpose+IntRate+Installment+LogAnnualInc+Dti+Fico+DaysWithCrLine+RevolBal+RevolUtil+InqLast6mths+Delinq2yrs+PubRec


In [8]:
model = smf.logit(col_str, data=df_loans_train)
model_results = model.fit()

print(model_results.summary())

Optimization terminated successfully.
         Current function value: 0.411580
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           NotFullyPaid   No. Observations:                 6704
Model:                          Logit   Df Residuals:                     6690
Method:                           MLE   Df Model:                           13
Date:                Fri, 17 Nov 2023   Pseudo R-squ.:                 0.06722
Time:                        17:36:56   Log-Likelihood:                -2759.2
converged:                       True   LL-Null:                       -2958.1
Covariance Type:            nonrobust   LLR p-value:                 6.925e-77
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          7.6790      1.523      5.042      0.000       4.694      10.664
CreditPolicy     

In [9]:
#Predict against training data
NFP_prob = model_results.predict(df_loans_train)

# NFP = NotFullyPaid
# Find NFP_prob = P(NFP = 1)
# Assume IF P(NFP=1) > 0.5 THEN Loan will not be paid
        #IF P(NFP=1) < 0.5 THEN loan will be paid

loan_paid = NFP_prob < 0.5
loan_debt = NFP_prob > 0.5

confusion_matrix = pd.crosstab(df_loans_train['NotFullyPaid'], loan_paid)
print(confusion_matrix)

col_0         False  True 
NotFullyPaid              
0                33   5592
1                24   1055


In [10]:
TP = 5592
TN = 24
FN = 33
FP = 1055
N_sum = TP+TN+FN+FP

In [11]:
#Model Accuracy = (TP +TN) / N_sum

model_acc = (TP+TN)/(N_sum)
print(model_acc)

0.837708830548926


# a_iii
Consider two applications, which are identical except App_A has FICO = 700, and App_B has FICO = 710.

Let Logit(A) be the value of the linear logit function of loan A NotFullyPaid.
Define Logit(B) similarly for loan B.

What is Logit(A) - Logit(B)?
> 0.0779



In [12]:
# From previous model:
int_coef = 7.6790
fico_coef = -0.0078

# Logit = b0 + b1_x1

# Logit(A)
fico_a = 700
logit_a = int_coef + fico_a*fico_coef
print(logit_a)

# Logit(B)
fico_b = 710
logit_b = int_coef + fico_b*fico_coef
print(logit_b)

print(logit_a - logit_b)


2.2190000000000003
2.141000000000001
0.0779999999999994


# a_iv

Now predict the probability of the test set of loans not being paid in full. Store these predicted probabilities in a variable **'PredictedRisk'** and add it to your test set.

What is the accuracy of the logistic regression model on test set using threshold of 0.5?
> 84.10%

How does this compare to baseline model?
>Baseline was 84.20%. This model is slightly worse by 0.10%


In [13]:
# Predict against test data
NFP_prob_testdata = model_results.predict(df_loans_test)

loan_paid = NFP_prob_testdata < 0.5
loan_debt = NFP_prob_testdata > 0.5

confusion_matrix = pd.crosstab(df_loans_test['NotFullyPaid'], loan_paid)
print(confusion_matrix)

col_0         False  True 
NotFullyPaid              
0                12   2408
1                 9    445


In [14]:
# Store the prob as 'PredictedRisk'

df_loans_test['PredictedRisk'] = NFP_prob_testdata

#Confirm new column
df_loans_test.head()


,CreditPolicy,Purpose,IntRate,Installment,LogAnnualInc,Dti,Fico,DaysWithCrLine,RevolBal,RevolUtil,InqLast6mths,Delinq2yrs,PubRec,NotFullyPaid,PredictedRisk
3409,1,0,0.1442,343.83,11.002100,3.68,667,14133.000000,6383,30.1,1,2,0,0,0.154846
8289,0,2,0.1229,50.03,10.308953,14.12,662,660.041667,669,7.5,1,0,0,0,0.189923
9293,0,2,0.1461,103.43,10.491274,22.13,667,2940.000000,37617,89.4,0,0,0,0,0.232585
3839,1,1,0.0932,159.74,11.050890,13.79,737,3630.000000,5163,28.1,1,0,0,0,0.065019
9050,0,4,0.1663,115.28,8.987197,6.30,707,660.000000,0,0.0,2,0,0,0,0.356151


In [15]:
#Model Accuracy = (TP +TN) / N_sum
TP = 2408
TN = 9
FP = 445
FN = 12

test_predict_acc = (TP+TN)/(TP+TN+FP+FN)
print(test_predict_acc)

0.8409881697981907


# a_v

What is the test set AUC of the model?
> 0.65

Given the accuracy and AUC on test set, do you think this model could be useful to an investory to make profitable investments?
> I would advise using this model as it is very risk averse. We see that recall is nearly 100% in identifying risky loans. From an investor's perspective this is a safe model.

In [16]:
# use roc_auc_score
roc_auc_score(df_loans_test['NotFullyPaid'], df_loans_test['PredictedRisk'])

0.6549714202497543

In [17]:
# Precision Metric
    #   TP/(TP+FP)
    #   ability to correctly identify class 1 among the predicted (these are column calculations)
precision = TP/(TP+FP)
print(precision)

# Recall
    #   TP/(TP+FN)
    #   ability to correctly identify something among that actually exists (these are row calculations)
recall = TP/(TP+FN)
print(recall)

f1_score = 2*recall*precision/(recall+precision)
print(f1_score)

0.8440238345601122
0.9950413223140496
0.9133320690309122


# b)
LendingClub assigns the interest rate to a loan based on their estimate of that loan's risk. This variable, **'IntRate'** is an indepedent variable in our dataset.

In this part we investigate just using the loans interest rate as a "smart baseline" to order the loans accoriding to risk

# b_i

Using the training set, build a logistic regression model that predicts the dependent variable **NotFullyPaid** using **IntRate** as the only indepdent variable.

Is IntRate significant in this model?
> Yes. P>|z| is = 0.

Was it significant in the first logistic regression model you built?
> Yes, but not as significant as the other independent variables.

How would you explain this difference?
> A single, simple logistic regression model produces a yes or no answer where as multivariate models provides probability based on independent variables.


In [18]:
model_intrate = smf.logit('NotFullyPaid ~ IntRate', data=df_loans_train)
model_results_intrate = model_intrate.fit()

print(model_results_intrate.summary())

Optimization terminated successfully.
         Current function value: 0.427025
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           NotFullyPaid   No. Observations:                 6704
Model:                          Logit   Df Residuals:                     6702
Method:                           MLE   Df Model:                            1
Date:                Fri, 17 Nov 2023   Pseudo R-squ.:                 0.03221
Time:                        17:36:56   Log-Likelihood:                -2862.8
converged:                       True   LL-Null:                       -2958.1
Covariance Type:            nonrobust   LLR p-value:                 2.370e-43
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.8200      0.168    -22.686      0.000      -4.150      -3.490
IntRate       17.0934      1.

In [19]:
# Correlation Matrix
corr = df_loans_train.corr()

#Apply heatmap
corr.style.background_gradient(vmin = 0, vmax = 1)

,CreditPolicy,Purpose,IntRate,Installment,LogAnnualInc,Dti,Fico,DaysWithCrLine,RevolBal,RevolUtil,InqLast6mths,Delinq2yrs,PubRec,NotFullyPaid
CreditPolicy,1.000000,-0.019332,-0.291884,0.047615,0.028924,-0.092240,0.338114,0.100385,-0.197355,-0.107751,-0.531647,-0.076343,-0.049123,-0.157570
Purpose,-0.019332,1.000000,-0.076988,-0.119135,-0.012315,-0.204358,0.152963,-0.006684,-0.017474,-0.242639,0.066728,-0.003130,-0.020718,0.041748
IntRate,-0.291884,-0.076988,1.000000,0.283605,0.058441,0.214974,-0.720840,-0.127703,0.094677,0.468537,0.207613,0.152605,0.102243,0.168616
Installment,0.047615,-0.119135,0.283605,1.000000,0.447286,0.048414,0.078912,0.181014,0.229450,0.077421,0.006513,-0.001173,-0.025054,0.058008
LogAnnualInc,0.028924,-0.012315,0.058441,0.447286,1.000000,-0.059817,0.111067,0.334421,0.366397,0.052256,0.025398,0.020203,0.023490,-0.039216
Dti,-0.092240,-0.204358,0.214974,0.048414,-0.059817,1.000000,-0.242465,0.056527,0.170507,0.330180,0.030753,-0.017384,0.002224,0.032918
Fico,0.338114,0.152963,-0.720840,0.078912,0.111067,-0.242465,1.000000,0.265899,-0.021689,-0.548775,-0.183430,-0.213879,-0.144169,-0.155927
DaysWithCrLine,0.100385,-0.006684,-0.127703,0.181014,0.334421,0.056527,0.265899,1.000000,0.219154,-0.025100,-0.052753,0.074616,0.068426,-0.036998
RevolBal,-0.197355,-0.017474,0.094677,0.229450,0.366397,0.170507,-0.021689,0.219154,1.000000,0.195773,0.027944,-0.032802,-0.027137,0.055901
RevolUtil,-0.107751,-0.242639,0.468537,0.077421,0.052256,0.330180,-0.548775,-0.025100,0.195773,1.000000,-0.010282,-0.039675,0.066043,0.087482


# b_ii

Use the model you just built to make predictions for the observations in the test set.

What is the highest predicted probability of a loan not being paid back in fullt on test set?
> Prob.max() = 45.15%

How many loans would we predict would not be paid back in full if we used a threshold of 0.5 to make predictions?
> 0

In [20]:
# Predict against test set
NFP_intrate_test = model_results_intrate.predict(df_loans_test)

#Store in new column
df_loans_test['PredRisk_IntRateOnly'] = NFP_intrate_test

print(NFP_intrate_test.max())

0.4515366238129087


In [21]:
#
loan_paid_test = NFP_intrate_test < 0.5
loan_debt_test = NFP_intrate_test > 0.5

confusion_matrix_test = pd.crosstab(df_loans_test['NotFullyPaid'], loan_paid_test)
print(confusion_matrix_test)

col_0         True
NotFullyPaid      
0             2420
1              454


In [22]:
df_loans_test.head()

,CreditPolicy,Purpose,IntRate,Installment,LogAnnualInc,Dti,Fico,DaysWithCrLine,RevolBal,RevolUtil,InqLast6mths,Delinq2yrs,PubRec,NotFullyPaid,PredictedRisk,PredRisk_IntRateOnly
3409,1,0,0.1442,343.83,11.002100,3.68,667,14133.000000,6383,30.1,1,2,0,0,0.154846,0.205037
8289,0,2,0.1229,50.03,10.308953,14.12,662,660.041667,669,7.5,1,0,0,0,0.189923,0.151975
9293,0,2,0.1461,103.43,10.491274,22.13,667,2940.000000,37617,89.4,0,0,0,0,0.232585,0.210382
3839,1,1,0.0932,159.74,11.050890,13.79,737,3630.000000,5163,28.1,1,0,0,0,0.065019,0.097364
9050,0,4,0.1663,115.28,8.987197,6.30,707,660.000000,0,0.0,2,0,0,0,0.356151,0.273420


# b_iii

Compute the test set AUC of the model.

How does this compare to the model using all the independent variables?
> AUC = 0.60 which is less than the model with all ind. variables (AUC_all = 0.65).

In your opinion which model is stronger?
> Accuracy is stronger at 84.20% but with threshold set to 0.5, the AUC is too low and does not give me confidence in this model.

> I would continue to fine-tune the threshold value for this model, in order to prove significance of IntRate alone as a predictor in loans NotFullyPaid

> Additionally, this model is slight more risk prone than the previous model. From an investor's persepective, I would use the previous model

In [23]:
roc_auc_score(df_loans_test['NotFullyPaid'], df_loans_test['PredRisk_IntRateOnly'])

0.6033831506899189

In [24]:
# Accuracy TP / N_sum
TP_test = 2420
FP_test = 454
N_test = TP+FP
acc_intrate_test = TP_test/N_test
print(acc_intrate_test)

0.848229933403435


In [25]:
# Precision Metric
    #   TP/(TP+FP)
    #   ability to correctly identify class 1 among the predicted (these are column calculations)
precision_test = TP_test/(TP_test+FP_test)
print(precision_test)

# Recall
    #   TP/(TP+FN)
    #   ability to correctly identify something among that actually exists (these are row calculations)
recall_test = TP_test/(TP_test)
print(recall_test)

f1_score_test = 2*recall_test*precision_test/(recall_test+precision_test)
print(f1_score_test)

0.8420320111343076
1.0
0.9142425387230827


# c

Let us now see how our logistic regression model can be used to identify loans that are expected to be profitable.


# c_i

If the loan is paid back in full, then the investor makes interest on the loan. However, if the loan is not paid back, the investor loses money invested. Therefore, investor should seek loans that best balance risk vs reward.

To compute interest revenue consider an investment, 'c', with annual interest rate, 'r', over a period of 't' years.
Compound interest = c * exp ** rt

How much does a $10 investment with r = 6% after 3 years pay?
> 11.97

In [26]:

c = 10
r = 0.06
t = 3

# Revenue
Revenue = c*(np.exp(r*t))
print(Revenue)

11.972173631218102


# c_ii

While the investment has this value after collecting interest, that investor had to pay c dollars for the investment.

What is the profit to the investor if the investment is paid back in full?
> 11.97 - 10 = 1.97 profit

What is the profit to the investor if the investment if not paid back in full?
> It is assumed that the investor would lose his money if loan is not paid back in full so there is no profit.



# c_iii

Compute the profit of a $1 dollar investment in each loan, and save your result to a variable named 'Profit'.

Keep in mind that the profit computation should depend on the value of the variable 'NotFullyPaid'

What is the maximum profit of a $1 investment in any loan in the testing set?
> 0.89


In [27]:
# We only care about loans that are paid back, so slice test set such that 'NotFullyPaid' == 0.
NFP_0 = df_loans_test[df_loans_test['NotFullyPaid'] == 0]

#Revenue = c*np.exp(r*t)
# c = $1, r = IntRate t = 3
c = 1
r = NFP_0['IntRate']
t = 3

#Profit = Revenue - principal
df_loans_test['Profit'] = (c*np.exp(r*t)) - c

#Show newly added column 'Profit'
print(df_loans_test.columns)

#Get max value
print(df_loans_test['Profit'].max())

Index(['CreditPolicy', 'Purpose', 'IntRate', 'Installment', 'LogAnnualInc',
       'Dti', 'Fico', 'DaysWithCrLine', 'RevolBal', 'RevolUtil',
       'InqLast6mths', 'Delinq2yrs', 'PubRec', 'NotFullyPaid', 'PredictedRisk',
       'PredRisk_IntRateOnly', 'Profit'],
      dtype='object')
0.8894768654675331


# c_iv

A simple investment strategy of equally investing in all the loans would yield a profit 20.94 for a $100 investment. Instead, let us analyze an investment strategy in which the investor only purchases loans with IntRate >= 15% to maximize return, but select loans with lowest predicted risk.

We will model an investor who invests $1 in each of the most promising 100 loans.

First, create a new dataset called 'HighInterest' consisting of test set loans with IntRate >= 15%.

What is the average profit of a $1 investment in one of these high-interest loans?
> 0.64

What proportion of the high-interest loans were not paid in full?
> 22.86%

In [28]:
#Create new dataset called 'HighInterest'
#Slice the test data set such that IntRate >= 0.15
df_HighInterest = df_loans_test[df_loans_test['IntRate'] >= 0.15]

print(df_HighInterest['Profit'].mean())

0.6383449116249343


In [29]:
#Find number of entries in high interest data where NotFullyPaid == 1
NFP_1_HighInt = len(df_HighInterest[df_HighInterest['NotFullyPaid'] == 1])

#Find number of entires in high interest data where NotFullyPaid == 0
NFP_0_HighInt = len(df_HighInterest[df_HighInterest['NotFullyPaid'] == 0])

# Take proportion of loans not paid in full (NFP_1 / NFP_0 + NFP_1)
print(NFP_1_HighInt / (NFP_0_HighInt + NFP_1_HighInt))

0.228643216080402


# c_v

Next, sort the loans in the HighInterest dataset by the variable 'PredictedRisk' that we computed earlier.

Create a new dataset called 'SelectedLoans' that consists of the 100 loans with the smallest values of PredictedRisk.

What is the profit of an investor who invested $1 in each of these 100 loans?
> 0.62

How many of the 100 selected loans were not paid back in full?
> 16%

How does this compare to the simple strategy of investing in all loans, which yieled a profit of 20.94 for a $100 investment?
> The return on investment (ROI) for the simple model is 20.94
while the ROI for the selected model is 62.0.
>> The selective strategy has a 3x better probability of yielding positive profits.





In [30]:
#From df_HighInterest we want only the 100 with lowest 'PredictedRisk'
df_SelectedLoans = df_HighInterest.nsmallest(100, ['PredictedRisk'])

#Calculate avg profit from profit column, since we already calculated this from last problem.
print(df_SelectedLoans['Profit'].mean())


0.6172046111413271


In [31]:
#Find number of entries in high interest data where NotFullyPaid == 1
NFP_1_HighInt_Select = len(df_SelectedLoans[df_SelectedLoans['NotFullyPaid'] == 1])

#Find number of entires in high interest data where NotFullyPaid == 0
NFP_0_HighInt_Select = len(df_SelectedLoans[df_SelectedLoans['NotFullyPaid'] == 0])

# Take proportion of loans not paid in full (NFP_1 / NFP_0 + NFP_1)
print(NFP_1_HighInt_Select / (NFP_0_HighInt_Select + NFP_1_HighInt_Select))

0.16


In [32]:
#ROI = Profit/Investment * 100%

#ROI for simple model
ROI_simple = 20.94/100 * 100

#ROI for selected
ROI_select = 0.62/1*100

print(f'''The return on investment (ROI) for the simple model is {ROI_simple}
while the ROI for the selected model is {ROI_select}.''' )



The return on investment (ROI) for the simple model is 20.94
while the ROI for the selected model is 62.0.


#d

One of the most important assumptions of predictive modeling often does not hold in financial situations, causing predictive models to fail.

What do you think this is?
> Predictive models are based on historical data, the assumption that the past will predict the future does not resonate well with financial institutions.

As an anaylst, what could you do to improve the situation?
> I believe the crux in financial modeling comes from human behavior/psychology. Humans, while reliable for the most part, are not immune to flaws or accidents. If we wish to improve our financial models we should try to find methods of extracting data from human behavior on a global scale. I believe these atttributes are the missing variable(s) which would allow our models to be truly predictive.